<a href="https://colab.research.google.com/github/Mozzer2310/text-mining-cwk/blob/sam-experiments/bert-experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
from datasets import load_dataset

dataset = load_dataset("dataset-org/dialog_re", download_mode="force_redownload", trust_remote_code=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.45k [00:00<?, ?B/s]

dialog_re.py:   0%|          | 0.00/4.83k [00:00<?, ?B/s]

dialog_re.py:   0%|          | 0.00/4.83k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.45k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1073 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/358 [00:00<?, ? examples/s]

In [2]:
dataset['test'][0]

{'dialog': ['Speaker 1: Hey, you guys! Look what I found! Look at this!  That’s my Mom’s writing! Look.',
  'Speaker 2: Me and Frank and Phoebe, Graduation 1965.',
  "Speaker 1: Y'know what that means?",
  'Speaker 3: That you’re actually 50?',
  'Speaker 1: No-no, that’s not, that’s not me Phoebe, that’s her pal Phoebe. According to her high school yearbook, they were like B.F.F. Best Friends Forever.',
  'Speaker 4: Oh!',
  'Speaker 5: That is so cool.',
  'Speaker 1: I know! So this woman probably could like have all kinds of stories about my parents, and she might even know like where my Dad is. So I looked her up, and she lives out by the beach. So maybe this weekend we could go to the beach?',
  'Speaker 4: Yeah! Yeah, we can!',
  'Speaker 6: Shoot! I can’t go, I have to work!',
  'Speaker 7: That’s too bad.',
  'Speaker 5: Ohh, big, fat bummerrr.',
  'Speaker 1: So great! Okay! Tomorrow we’re gonna drive out to Montauk.'],
 'relation_data': {'x': ['Speaker 1',
   'Speaker 1',
  

**Data Fields**
- `dialog`
    - List of dialog spoken between the speakers
- List of annotations per dialog per argument
    - `x` : First entity
    - `y` : Second entity
    - `x_type` : Type of the first entity
    - `y_type`: Type of the second entity
    - `r` : List of relations
    - `rid`: List of relation IDs
    - `t`: List of relation Trigger words

In [9]:
import re
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [77]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [78]:
def add_tokens(example, add_triggers=True):
    """Converts datapoint to a list of tuples containing dialog with separated entities and their relation"""

    dialog = example['dialog']
    relation_data = example['relation_data']

    data = []
    relation_labels = []

    # Join the dialog into a single string
    all_dialog = ' '.join(dialog)

    for x, y, r, t in zip(relation_data['x'], relation_data['y'], relation_data['r'], relation_data['t']):
        # Create a dictionary to store the formatted dialog
        formatted_text = f"{all_dialog} [SEP] {x} [SEP] {y}"

        # optionally include trigger words
        if add_triggers:
            triggers = ', '.join(t)
            formatted_text = f"{formatted_text} [SEP] {triggers}"

        data.append(formatted_text)
        relation_labels.append(r)

    return data, relation_labels

In [79]:
add_tokens(dataset['test'][0])

(["Speaker 1: Hey, you guys! Look what I found! Look at this!  That’s my Mom’s writing! Look. Speaker 2: Me and Frank and Phoebe, Graduation 1965. Speaker 1: Y'know what that means? Speaker 3: That you’re actually 50? Speaker 1: No-no, that’s not, that’s not me Phoebe, that’s her pal Phoebe. According to her high school yearbook, they were like B.F.F. Best Friends Forever. Speaker 4: Oh! Speaker 5: That is so cool. Speaker 1: I know! So this woman probably could like have all kinds of stories about my parents, and she might even know like where my Dad is. So I looked her up, and she lives out by the beach. So maybe this weekend we could go to the beach? Speaker 4: Yeah! Yeah, we can! Speaker 6: Shoot! I can’t go, I have to work! Speaker 7: That’s too bad. Speaker 5: Ohh, big, fat bummerrr. Speaker 1: So great! Okay! Tomorrow we’re gonna drive out to Montauk. [SEP] Speaker 1 [SEP] 50 [SEP] ",
  "Speaker 1: Hey, you guys! Look what I found! Look at this!  That’s my Mom’s writing! Look. S

In [80]:
test_data = []
test_relation_labels = []
for datapoint in dataset['test']:
    data, relation_labels = add_tokens(datapoint)
    test_data.extend(data)
    test_relation_labels.extend(relation_labels)

print(test_data[0])
print(test_relation_labels[0])

Speaker 1: Hey, you guys! Look what I found! Look at this!  That’s my Mom’s writing! Look. Speaker 2: Me and Frank and Phoebe, Graduation 1965. Speaker 1: Y'know what that means? Speaker 3: That you’re actually 50? Speaker 1: No-no, that’s not, that’s not me Phoebe, that’s her pal Phoebe. According to her high school yearbook, they were like B.F.F. Best Friends Forever. Speaker 4: Oh! Speaker 5: That is so cool. Speaker 1: I know! So this woman probably could like have all kinds of stories about my parents, and she might even know like where my Dad is. So I looked her up, and she lives out by the beach. So maybe this weekend we could go to the beach? Speaker 4: Yeah! Yeah, we can! Speaker 6: Shoot! I can’t go, I have to work! Speaker 7: That’s too bad. Speaker 5: Ohh, big, fat bummerrr. Speaker 1: So great! Okay! Tomorrow we’re gonna drive out to Montauk. [SEP] Speaker 1 [SEP] 50 [SEP] 
['per:age']


In [82]:
test_encodings = tokenizer(test_data, padding="max_length", truncation=True)
print(test_encodings[0])

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [83]:
print(tokenizer.decode(test_encodings[0].ids))

['per:age']
[CLS] speaker 1 : hey, you guys! look what i found! look at this! that ’ s my mom ’ s writing! look. speaker 2 : me and frank and phoebe, graduation 1965. speaker 1 : y ' know what that means? speaker 3 : that you ’ re actually 50? speaker 1 : no - no, that ’ s not, that ’ s not me phoebe, that ’ s her pal phoebe. according to her high school yearbook, they were like b. f. f. best friends forever. speaker 4 : oh! speaker 5 : that is so cool. speaker 1 : i know! so this woman probably could like have all kinds of stories about my parents, and she might even know like where my dad is. so i looked her up, and she lives out by the beach. so maybe this weekend we could go to the beach? speaker 4 : yeah! yeah, we can! speaker 6 : shoot! i can ’ t go, i have to work! speaker 7 : that ’ s too bad. speaker 5 : ohh, big, fat bummerrr. speaker 1 : so great! okay! tomorrow we ’ re gonna drive out to montauk. [SEP] speaker 1 [SEP] 50 [SEP] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

**Need to encode label data**  
Use a binary multi-hot vector if multiple labels

In [103]:
def get_labels(dataset):
    all_dataset_labels = set()
    for datapoint in dataset['train']:
        for relation in  [item for sublist in datapoint['relation_data']['r'] for item in sublist]:
            all_dataset_labels.add(relation)
    return list(all_dataset_labels)

In [104]:
from sklearn.preprocessing import MultiLabelBinarizer

# Get all possible labels
all_labels = get_labels(dataset)
print(all_labels)

# Initialize the binarizer
mlb = MultiLabelBinarizer(classes=all_labels)

# Fit and transform the labels
encoded_labels = mlb.fit_transform(test_relation_labels)

['per:date_of_birth', 'per:acquaintance', 'org:students', 'per:place_of_work', 'per:girl/boyfriend', 'unanswerable', 'per:pet', 'per:other_family', 'per:visited_place', 'per:client', 'per:place_of_residence', 'org:employees_or_members', 'per:schools_attended', 'per:title', 'per:boss', 'per:subordinate', 'per:origin', 'gpe:residents_of_place', 'per:roommate', 'per:children', 'per:alumni', 'gpe:visitors_of_place', 'per:major', 'per:alternate_names', 'per:employee_or_member_of', 'per:neighbor', 'per:negative_impression', 'per:age', 'per:parents', 'per:positive_impression', 'per:siblings', 'per:dates', 'per:friends', 'per:spouse', 'per:works']


In [112]:
print(encoded_labels[0])
print(mlb.classes_)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
['per:date_of_birth' 'per:acquaintance' 'org:students' 'per:place_of_work'
 'per:girl/boyfriend' 'unanswerable' 'per:pet' 'per:other_family'
 'per:visited_place' 'per:client' 'per:place_of_residence'
 'org:employees_or_members' 'per:schools_attended' 'per:title' 'per:boss'
 'per:subordinate' 'per:origin' 'gpe:residents_of_place' 'per:roommate'
 'per:children' 'per:alumni' 'gpe:visitors_of_place' 'per:major'
 'per:alternate_names' 'per:employee_or_member_of' 'per:neighbor'
 'per:negative_impression' 'per:age' 'per:parents'
 'per:positive_impression' 'per:siblings' 'per:dates' 'per:friends'
 'per:spouse' 'per:works']
